<div align="center">
    <img src="https://socialify.git.ci/julep-ai/julep/image?description=1&descriptionEditable=Build%20AI%20agents%20and%20workflows%20with%20a%20simple%20API&font=Source%20Code%20Pro&logo=https%3A%2F%2Fraw.githubusercontent.com%2Fjulep-ai%2Fjulep%2Fdev%2F.github%2Fjulep-logo.svg&owner=1&pattern=Solid&stargazers=1&theme=Auto" alt="julep" width="640" height="320" />
</div>

## Task Definition: Spider Crawler Integration

### Overview

This task is a simple task that leverages the spider `integration` tool, and combines it with a prompt step to crawl a website for a given URL, and then create a summary of the results.

### Task Tools:

**Spider Crawler**: An `integration` type tool that can crawl the web and extract data from a given URL.

### Task Input:

**url**: The URL of the website to crawl.

### Task Output:

**output**: A dictionary that contains a `documents` key which contains the extracted data from the given URL. Check the output below for a detailed output schema.

### Task Flow

1. **Input**: The user provides a URL to crawl.

2. **Spider Tool Integration**: The `spider_crawler` tool is called to crawl the web and extract data from the given URL.

3. **Prompt Step**: The prompt step is used to create a summary of the results from the spider tool.

4. **Output**: The final output is the summary of the results from the spider tool.

## Implementation

To recreate the notebook and see the code implementation for this task, you can access the Google Colab notebook using the link below:

<a target="_blank" href="https://colab.research.google.com/github/julep-ai/julep/blob/dev/cookbooks/01-Website_Crawler_using_Spider.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Additional Information

For more details about the task or if you have any questions, please don't hesitate to contact the author:

**Author:** Julep AI  
**Contact:** [hey@julep.ai](mailto:hey@julep.ai) or  <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>

Installing the Julep Client

In [10]:
!pip install julep -U --quiet

In [1]:
import uuid

# NOTE: these UUIDs are used in order not to use the `create_or_update` methods instead of
# the `create` methods for the sake of not creating new resources every time a cell is run.
AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

### Creating Julep Client with the API Key

In [2]:
from julep import Client
import os

api_key = os.getenv("JULEP_API_KEY")

# Create a Julep client
client = Client(api_key=api_key, environment="dev")

## Creating an "agent"

Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the [documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#agent).

In [3]:
# Create agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name="Spiderman",
    about="AI that can crawl the web and extract data",
    model="gpt-4o",
)

## Defining a Task

Tasks in Julep are Github-Actions-style workflows that define long-running, multi-step actions.

You can use them to conduct complex actions by defining them step-by-step.

To learn more about tasks, please refer to the `Tasks` section in [Julep Concepts](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#tasks).

In [46]:
import yaml

spider_api_key = os.getenv("SPIDER_API_KEY")

# Define the task
task_def = yaml.safe_load(f"""
name: Crawling Task

# Define the tools that the agent will use in this workflow
tools:
- name: spider_crawler
  type: integration
  integration:
    provider: spider
    setup:
      spider_api_key: "{spider_api_key}"

# Define the steps of the workflow
main:
# Define a tool call step that calls the spider_crawler tool with the url input
- tool: spider_crawler
  arguments:
    url: "_['url']" # You can also use 'inputs[0]['url']'
  
    
- prompt: |
    You are {{{{agent.about}}}}
    I have given you this url: {{{{inputs[0]['url']}}}}
    And you have crawled that website. Here are the results you found:
    {{{{_['documents']}}}}
    I want you to create a short summary (no longer than 100 words) of the results you found while crawling that website.

  unwrap: True
""")

<span style="color:yellow;">Notes:</span>
- The reason for using the quadruple curly braces `{{{{}}}}` for the jinja template is to avoid conflicts with the curly braces when using the `f` formatted strings in python. [More information here](https://stackoverflow.com/questions/64493332/jinja-templating-in-airflow-along-with-formatted-text)
- The `unwrap: True` in the prompt step is used to unwrap the output of the prompt step (to unwrap the `choices[0].message.content` from the output of the model).


## Creating/Updating a task

In [47]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

## Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

Creates a execution worklow for the Task defined in the yaml file.

In [48]:
# creating an execution object
execution = client.executions.create(
    task_id=TASK_UUID,
    input={
        "url": "https://spider.cloud"
    }
)

There are multiple ways to get the execution details and the output:

1. **Get Execution Details**: This method retrieves the details of the execution, including the output of the last transition that took place.

2. **List Transitions**: This method lists all the task steps that have been executed up to this point in time, so the output of a successful execution will be the output of the last transition (first in the transition list as it is in reverse chronological order), which should have a type of `finish`.


<span style="color:yellow;">Note: You need to wait for a few seconds for the execution to complete before you can get the final output, so feel free to run the following cells multiple times until you get the final output.</span>


In [53]:
# Get execution details
execution = client.executions.get(execution.id)
# Print the output
print(execution.output)

Spider.cloud is a cutting-edge web crawling service designed for AI applications, offering high-speed and cost-effective data collection. Built with a robust Rust engine, Spider supports seamless integration with AI tools and provides various response formats, including LLM-ready markdown. It features concurrent streaming, caching, smart mode with headless Chrome, and auto proxy rotations. Ideal for large-scale projects, Spider ensures compliance with robots.txt and offers a free trial. Trusted by tech leaders, it enables efficient data curation and transformation, with capabilities to handle extreme workloads and dynamic content rendering.


In [54]:
# Lists all the task steps that have been executed up to this point in time
transitions = client.executions.transitions.list(execution_id=execution.id).items

# Transitions are retreived in reverse chronological order
for transition in reversed(transitions):
    print("Transition type: ", transition.type)
    print("Transition output: ", transition.output)
    print("-"*50)

Transition type:  init
Transition output:  {'url': 'https://spider.cloud'}
--------------------------------------------------
Transition type:  step
Transition output:  {'documents': [{'id': None, 'metadata': {'description': 'Experience cutting-edge web crawling with unparalleled speeds, perfect for LLMs, Machine Learning, and Artificial Intelligence. The fastest and most efficient web scraper tailored for AI applications.', 'domain': 'spider.cloud', 'extracted_data': None, 'file_size': 9112, 'keywords': ['AI agent stack', 'AI 🕷️ Spider', 'AWS infrastructure reduced', 'Auto Proxy rotations', 'Comprehensive Data Curation', 'Concurrent Streaming Save time', 'Data Collecting Projects Today Jumpstart web crawling', 'FAQ Frequently asked questions', 'Fastest Web Crawler', 'Multiple response formats', 'Open Source Spider engine', 'Performance Tuned Spider', 'Seamless Integrations Seamlessly integrate Spider', 'Smart Mode Spider dynamically switches', 'Spider accurately crawls', 'Spider conve

## Running the same task with a different URL

We will use the same code to run the same task, but with a different URL

In [59]:
execution = client.executions.create(
    task_id=TASK_UUID,
    input={
        "url": "https://www.harvard.edu/"
    }
)

In [66]:
execution = client.executions.get(execution.id)
print("\n".join(execution.output.split(". ")))

Harvard University's website highlights its commitment to excellence in teaching, research, and leadership development globally
It offers diverse academic programs, including undergraduate, graduate, and professional learning
The site features information about Harvard's various schools, libraries, museums, and research initiatives
It emphasizes Harvard's global impact, historical achievements, and contributions to fields like climate change, medicine, and biodiversity
The site also provides resources for campus visits, events, and news updates, showcasing Harvard's vibrant community and its role in advancing knowledge and societal well-being.


Note: you can get the output of the crawling step by accessing the corresponding transition's output from the transitions list.

Example:

In [67]:
transitions = client.executions.transitions.list(execution_id=execution.id).items

transitions[1].output

{'documents': [{'id': None,
   'metadata': {'description': 'Harvard University is devoted to excellence in teaching, learning, and research, and to developing leaders who make a difference globally.',
    'domain': 'www.harvard.edu',
    'extracted_data': None,
    'file_size': 19620,
    'keywords': ['Amazon rainforest immersion',
     'Ants Army ants',
     'Arnold Arboretum Horticultural Library',
     'Cabot Science Library',
     'Dumbarton Oaks Research Library',
     'Ernst Mayr Library',
     'Fine Arts Library',
     'Frances Loeb Library',
     'Griffin Graduate School',
     'Harvard Amazon Rainforest Immersion program',
     'Harvard Art Museums',
     'Harvard Business School',
     'Harvard Divinity School',
     'Harvard Divinity School Library',
     'Harvard Film Archive',
     'Harvard Forest research project explored naturalist Frank Morton Jones',
     'Harvard Graduate School',
     'Harvard Kennedy School',
     'Harvard Law School',
     'Harvard Law School Libra